### 상대모멘텀 
- 10개의 주식 데이터를 이용하여 투자 전략 테스팅
1. 월초부터 월말까지의 수정주가를 이용하여 월별 수익율
2. 월별 수익율이 높은 순서중 n개의 주식을 선택 
3. 해당하는 주식들을 매수 후 매도하여 수익율 계산

In [ ]:
import pandas as pd 
from datetime import datetime
import numpy as np
import os 
import glob
import warnings

In [ ]:
# warning message 제거 
warnings.filterwarnings('ignore')

In [ ]:
# 월별 수익율을 계산하는 함수 생성 
def create_1M_rtn(_df, _ticker, _start = "2010-01-01", _col = 'Adj Close'):
    # 복사본 생성 
    result = _df.copy()
    # 컬럼에 Date가 포함되어있는지 확인 
    if 'Date' in result.columns:
        result = result.loc[result['Date'] >= _start, ['Date', _col]]
        # Date 컬럼의 데이터를 시계열로 변경
        result['Date'] = pd.to_datetime(result['Date'], format='%Y-%m-%d')
        result.set_index('Date', inplace=True)
    else:
        result.index = pd.to_datetime(result.index, inplace=True)
        result = result.loc[_start:, [_col]]
    # 기준 년월 컬럼을 생성
    result['STD-YM'] = result.index.strftime('%Y-%m')
    result['1m_rtn'] = 0
    result['CODE'] = _ticker
    # 기준 년월의 중복데이터를 제거하고 고유한 값들을 리스트로 생성
    ym_list = result['STD-YM'].unique()
    return result, ym_list

In [ ]:
aapl = pd.read_csv('../../csv/AAPL.csv')

In [ ]:
sample_aapl, ym_list = create_1M_rtn(aapl, 'AAPL')

In [ ]:
sample_aapl.head()

In [ ]:
ym_list

In [ ]:
# 특정 경로에 있는 파일들의 목록을 로드 
os.listdir("./data")

files = glob.glob("./data/*.csv")
files

In [ ]:

# 새로운 데이터프레임을 생성 
# 종목별 데이터프레임 
stock_df = pd.DataFrame()
# 월말 데이터프레임 
month_last_df = pd.DataFrame()

for file in files:
    folder, name = os.path.split(file)
    # print(folder, name)
    head, tail = os.path.splitext(name)
    # print(head, tail)
    # head는 create_1M_rtn 함수에 ticker 인자값으로 사용

    # 데이터 파일을 로드 
    read_df = pd.read_csv(file)

    # create_1M_rtn 함수를 호출 
    price_df, ym_list = create_1M_rtn(read_df, head)

    # 유니언 결합 (단순한 행 결합 함수)
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 월별 상태 모멘텀을 계산하기 위해 1개월간의 수익율 계산
    for ym in ym_list:
        flag = price_df['STD-YM'] == ym
        m_rtn = price_df.loc[flag,].iloc[-1, 0] / price_df.loc[flag,].iloc[0, 0]
        price_df.loc[flag, '1m_rtn'] = m_rtn
        data = price_df.loc[flag, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat([month_last_df, data], axis=0)

In [ ]:
month_last_df.tail()

In [ ]:
month_rtn_df = month_last_df.copy()

In [ ]:
month_rtn_df.reset_index(inplace=True)

In [ ]:
month_rtn_df.head()

In [ ]:
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns= 'CODE', 
    values= '1m_rtn'
)

In [ ]:
month_rtn_df.head()

In [ ]:
month_rtn_df = month_rtn_df.rank(
    axis=1, 
    ascending=False, 
    method='max', 
    pct=True
)

In [ ]:
month_rtn_df

In [ ]:
# 상위 15% 종목만 선택 
# where(조건식, 거짓일때 변경될 데이터)
month_rtn_df =  month_rtn_df.where(month_rtn_df < 0.15, 0)

In [ ]:
month_rtn_df.head()

In [ ]:
# 데이터 중에서 0이 아닌 데이터는 1로 변경 
month_rtn_df[month_rtn_df != 0] = 1

In [ ]:
month_rtn_df

In [ ]:
# CODE 값들을 유니크 데이터만 따로 출력 
stock_codes = stock_df['CODE'].unique()
stock_codes

In [ ]:
month_rtn_df.loc["2010-01-29 00:00:00", month_rtn_df.loc["2010-01-29 00:00:00"] >= 1].index

In [ ]:
sig_dict = dict()

for date in month_rtn_df.index:
    # print(date)
    ticker_list = list(
        month_rtn_df.loc[date, month_rtn_df.loc[date] >= 1].index)
    # print(ticker_list)
    sig_dict[date] = ticker_list

In [ ]:
sig_dict

In [ ]:
# 거래 내역컬럼을 생성하는 데이터프레임 생성하는 함수 
def create_trade_book(_df, _code):
    book = _df[_code].copy()
    book['STD-YM'] = book.index.strftime('%Y-%m')
    for c in _code:
        book['p'+c] = ""
        book['r'+c] = ""
    return book

In [ ]:
stock_df.head()

In [ ]:
stock_c_matrix = stock_df.reset_index().pivot_table(
    index='Date', 
    columns = 'CODE', 
    values= stock_df.columns[0]
)

In [ ]:
stock_c_matrix

In [ ]:
book = create_trade_book(stock_c_matrix, stock_codes)

In [ ]:
book.columns

In [ ]:
sig_dict.items()

In [ ]:
# 포지션을 생성 
for date, values in sig_dict.items():
    # print(date, values)
    for stock in values:
        book.loc[date, 'p'+stock] = 'ready'+stock

In [ ]:
book[['pBND']].iloc[15:25]

In [ ]:
# 거래 내역 추가 
def trading(_book, s_code):
    std_ym = ""
    buy_phase = False

    # 종목별로 순회
    for code in s_code:
        for i in _book.index:
            # 해당 종목코드의 포지션을 잡는다.
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.shift().loc[i, 'p'+code] == "ready"+code):
                std_ym = book.loc[i, 'STD-YM']
                buy_phase = True
            # 해당 종목코드에서 신호가 잡혀있다면 매수 상태 유지 
            if (_book.loc[i, 'p'+code] == "") & \
                (_book.loc[i, 'STD-YM'] == std_ym) & \
                (buy_phase):
                _book.loc[i, 'p'+code] = 'buy'+code
            
            # std_ym, buy_phase 초기화
            if _book.loc[i, 'p'+code] == "":
                std_ym = ""
                buy_phase = False
    return _book
                

In [ ]:
book = trading(book, stock_codes)

In [ ]:
book['pAAPL'].value_counts()

In [ ]:
# 수익율 계산 함수 
def multi_returns(_book, s_code):
    rtn = 1
    buy_dict = dict()
    sell_dict = dict()

    for i in _book.index:
        for code in s_code:
            # 매수 (p+code 2일전에 "" 1일전에 ready 오늘이 buy)
            if (_book.shift(2).loc[i, "p"+code] == "") & \
                (_book.shift(1).loc[i, 'p'+code] == "ready"+code) & \
                (_book.loc[i, 'p'+code] == "buy"+code):
                buy_dict[code] = book.loc[i, code]
                print(f"매수일 : {i}, 종목코드 : {code}, 매수가 : {buy_dict[code]}")
            # 매도 (1일 전의 pcode가 buy 오늘의 pcode가 "")
            elif (_book.shift(1).loc[i, "p"+code] == "buy"+code) & \
                (_book.loc[i, 'p'+code] == ""):
                sell_dict[code] = _book.loc[i, code]
                # 수익율 계산 
                rtn = sell_dict[code] / buy_dict[code]
                _book.loc[i, 'r'+code] = rtn
                print(f"매도일 : {i}, 종목코드 : {code}, 매도가 : {sell_dict[code]}, 수익율 : {rtn}")
            # buy_dict, sell_dict 데이터를 초기화
            if _book.loc[i, 'p'+code] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
    return _book


In [ ]:
rtn_book = multi_returns(book, stock_codes)

In [ ]:
# 누적 수익율 계산 함수 
def multi_acc_returns(_book, s_code):
    # 누적 수익율 변수 생성
    acc_rtn = 1
    for i in _book.index:
        count = 0
        rtn = 0
        for code in s_code:
            # 수익율 데이터가 존재하는 경우
            if _book.loc[i, 'r'+code]:
                count += 1
                rtn += _book.loc[i, "r"+code]
        if (rtn != 0) & (count != 0):
            acc_rtn *= (rtn / count)
            print(f"누적 매도일 : {i}, 매도 종목수 : {count}, 수익율 : {round(rtn/count, 2)}")
        _book.loc[i, 'acc_rtn'] = acc_rtn
    
    return _book, acc_rtn

In [ ]:
multi_acc_returns(rtn_book, stock_codes)

In [ ]:
import momentum2 as mmt2

In [ ]:
import importlib

In [ ]:
importlib.reload(mmt2)

In [ ]:
# data_load({경로}) 
# 해당 경로에 있는 csv 파일을 모두 결합하여 
# (날짜,수정주가, 기준년월, 월별수익율, code값을 가진 데이터프레임), 
# (월말 데이터를 가지는 데이터프레임) 리턴 함수
stock_df, last_df = mmt2.data_load()

In [ ]:
stock_df.head()

In [ ]:
last_df.head()

In [ ]:
## create_position({월말 데이터프레임}, {상위 퍼센트})
## 리턴 값 : 구매 시기를 나타내는 딕셔너리 , 종목들의 리스트 
sig_dict, stock_codes = mmt2.create_position(last_df)

In [ ]:
stock_df.head()

In [ ]:
# create_trade_book({stock_df}, {종목 리스트}, {구매 시기 딕셔너리})
# 리턴 데이터 : 구매시기를 체크하여 구매 포지션을 지정한 데이터프레임
book = mmt2.create_trade_book(stock_df, stock_codes, sig_dict)

In [ ]:
# trading({book}, {종목 리스트})
# 구매 내역을 추가하여 데이터프레임 리턴
trade_book = mmt2.trading(book, stock_codes)

In [ ]:
# multi_returns({trade_book}, {종목 리스트})
# 수익율 데이터를 추가하여 데이터프레임을 리턴
rtn_book = mmt2.multi_returns(trade_book, stock_codes)

In [ ]:
# multi_acc_returns({rtn_book}, {종목 리스트})
# 누적수익율을 계산하여 데이터프레임에 대입한 결과와 총 누적 수익율을 리턴
acc_rtn_book, acc_rtn = mmt2.multi_acc_returns(rtn_book, stock_codes)